In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'domain': 'spouse',
    'postgres': False,
    'parallelism': 1,
    'debug': False,
    'splits': [0,1,2],
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_spouse.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

if config['debug']:
    print("NOTE: --debug=True: modifying parameters...")
    config['max_docs'] = 100
    config['gen_model_search_space'] = 2
    config['disc_model_search_space'] = 2
    config['gen_params_default']['epochs'] = 25
    config['disc_params_default']['n_epochs'] = 5

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

Overwriting domain=None to domain=spouse
Overwriting init_class_prior=0 to init_class_prior=-1.15
Overwriting reg_param=0.1 to reg_param=0.5
Overwriting decay=0.95 to decay=0.99


In [5]:
%time pipe.parse()

Clearing existing...
Running UDF...
Documents: 2591
Sentences: 67820
CPU times: user 7min 16s, sys: 10.9 s, total: 7min 27s
Wall time: 7min 29s


In [6]:
%time pipe.extract()

Loading predefined splits from file.
Separating sentences into splits.
Extracting candidates.
Clearing existing...
Running UDF...
[========================================] 100%

Candidates [Split 0]: 48177
Clearing existing...
Running UDF...
[========================================] 100%

Candidates [Split 1]: 7457
Clearing existing...
Running UDF...
[========================================] 100%

Candidates [Split 2]: 6101
CPU times: user 6min 42s, sys: 3.47 s, total: 6min 45s
Wall time: 6min 47s


In [7]:
%time pipe.load_gold()

AnnotatorLabels created: 0
AnnotatorLabels created: 2796
AnnotatorLabels created: 2697
CPU times: user 1min 54s, sys: 982 ms, total: 1min 55s
Wall time: 1min 55s


In [ ]:
%time pipe.collect()

In [ ]:
%time pipe.label()

In [ ]:
%time pipe.supervise()

In [ ]:
%time pipe.classify(config)